# Lattice design with Xsuite

We will uses as example the PIMM lattice developed by the TERA collaboration for proton and ion therapy ([CERN/PS 99-010](https://cds.cern.ch/record/385378/)) and implemented in the CNAO and MEDAUSTRON synchrotrons.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xtrack as xt

%config InlineBackend.figure_format = "retina"

### Build Environment and Define a Reference Particle

In [ ]:
env = xt.Environment()
env.particle_ref = xt.Particles(kinetic_energy0=200e6)  # 200MeV
env.vars.default_to_zero = True  # Undefined variables in env are set to zero.

### Define Elements

In [ ]:
# Element geometry
n_bends = 16
env["ang_mb"] = 2 * np.pi / n_bends
env["l_mb"] = 1.65
env["l_mq"] = 0.35

env.new("mb", parent=xt.RBend, length="l_mb", angle="ang_mb", k0_from_h=True)
env.new("mq", parent=xt.Quadrupole, length="l_mq");

In [ ]:
# Quadrupole families with different strengths
env.new("qfa", parent="mq", k1="kqfa")
env.new("qfb", parent="mq", k1="kqfb")
env.new("qd", parent="mq", k1="kqd");

### Build Lattice Cells

In [ ]:
cell_a = env.new_line(
    length=7.405,
    components=[
        env.place("qfa", at=0.3875),
        env.place("mb", at=1.8125),
        env.place("qd", at=3.2925),
        env.place("mb", at=5.0475),
        env.place("qfa", at=6.3275),
    ],
)
cell_a.survey().plot()

In [ ]:
# Similar cell with second familiar of focusing quads
cell_b = env.new_line(
    name="cell_b",
    length=8.405,
    components=[
        env.place("qfb", at=1.2725),
        env.place("mb", at=2.7275),
        env.place("qd", at=4.8575),
        env.place("mb", at=6.5125),
        env.place("qfb", at=7.7925),
    ],
)
# cell_b.survey().plot()

### Build an Arc

In [ ]:
# Concatenate the two cells
arc = cell_a + cell_b
# arc.survey().plot()

### Build Straight Sections

In [ ]:
long_straight = env.new_line(length=2.0, components=[env.new("mid.lss", parent=xt.Marker, at=1.0)])
short_straight = env.new_line(length=1.0, components=[env.new("mid.sss", parent=xt.Marker, at=0.5)])

### Assemble the Ring

In [ ]:
half_ring = long_straight + arc + short_straight - arc  # mirror symmetric lattice
# half_ring.survey().plot()

In [ ]:
ring = 2 * half_ring
ring.survey().plot()

### Replace Repeated Elements

In [ ]:
# Gives all elements unique names
ring.replace_all_repeated_elements()

### Inspect Beamline Table

In [ ]:
table = ring.get_table()
table.cols["element_type", "s_start", "s_center", "s_end"]

In [ ]:
# Inspect all quadrupoles
table_quads = table.rows[table.element_type == "Quadrupole"]
table_quads.cols["s_start", "s_center", "s_end"]

In [ ]:
# Tag all quadrupoles in survey plot
sv = ring.survey()
sv.plot(labels=table_quads.name);

### Define and Install Sextupoles

In [ ]:
# Magnet type
env.new("ms", parent=xt.Sextupole, length=0.2)

# Magnet instances
env.new("msf.1", parent="ms", k2="ksf")  # focusing
env.new("msf.2", parent="ms", k2="ksf")  # focusing
env.new("msd.1", parent="ms", k2="ksd")  # defocusing
env.new("msd.2", parent="ms", k2="ksd")  # defocusing
env.new("mse", parent="ms", k2="kse")  # for extraction

In [ ]:
ring.insert(
    [
        env.place("msf.1", at=-0.2, from_="qfb.0@start"),
        env.place("msf.2", at=-0.2, from_="qfb.4@start"),
        env.place("msd.1", at=0.3, from_="qd.2@end"),
        env.place("msd.2", at=0.3, from_="qd.6@end"),
        env.place("mse", at=-0.3, from_="qfa.4@start"),
    ]
)

In [ ]:
# Inspect sextupoles in the survey
survey = ring.survey()
survey.plot(labels=["msf.1", "msf.2", "msd.1", "msd.2", "mse"])

## Inspect Circuit Structure

In [ ]:
# Entities controlled by one knob
env.info("ksf")

In [ ]:
# Inspect knob controlling one magnet
env.info("msf.1")

### Install RF Cavity

In [ ]:
env.new("rf1", parent=xt.Cavity, voltage="vrf", frequency="frf")
ring.insert("rf1", at=0.5, from_="qfa.3@start")

In [ ]:
survey = ring.survey()
survey.plot(labels=["rf1"])

### Save lattice to json file

In [ ]:
env["ring"] = ring
env.to_json("pimm.json")

---